In [1]:
import cert.cert
import cert.cert.util
import cert.cert.ser
import cert.serve
from cryptography import x509
import itertools
from pathlib import Path
from urllib.parse import quote
import webbrowser
import json

In [3]:
# generate keys
certs: dict[int, tuple[Path, bytes]] = dict()

cn = cert.cert.simple_common_name("127.0.0.1")
priv_key = cert.cert.make_private_key()

local_dir = Path.cwd() / "local"

priv_key_path = local_dir / "experiment.key"
priv_key_path.write_bytes(cert.cert.ser.serialize_private(priv_key))

for i, args in enumerate(itertools.product((True, False), repeat=9)):
    try:
        restrictions = x509.KeyUsage(*args)
        rest_rep = repr(restrictions).encode()
    except ValueError:
        continue

    leaf_cert = cert.cert.sign_builder(
        cert.cert.CertBuilderArgs(
            subject=cn, issuer=cn, public_key=priv_key.public_key()
        ).make_builder(),
        priv_key,
    )

    cert_path = local_dir / f"{i}.pem"
    cert_path.write_bytes(cert.cert.ser.serialize_public_cert(leaf_cert))

    certs[i] = cert_path, rest_rep

In [5]:
# certs_n_servers: dict[int, tuple[bytes, cert.serve.Server]] = dict()
successes = list()
failures = list()

In [6]:
for i, (cert_path, rest_rep) in certs.items():
    server = cert.serve.serve(priv_key_path, cert_path, rest_rep)

    server.start()

    url = server.url() + "/" + quote(rest_rep)

    webbrowser.open(url, autoraise=False)

    while True:
        res = input("success? ")
        match res:
            case "y":
                successes.append(rest_rep)
                break
            case "n":
                failures.append(rest_rep)
                break
            case _:
                print("huh?")

    server.stop()


TypeError: 'NoneType' object is not callable